## **Data Preprocessing.**

#### Importing Some important Libraries.

In [1]:
import pandas as pd
import warnings
import numpy as np
import sklearn as sns
from sklearn.preprocessing import OneHotEncoder,LabelEncoder ,RobustScaler,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline 
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import roc_curve, auc
import pandas as pd
import pickle
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

#### Importing data.

In [2]:
df = pd.read_csv(r'Sleep_health_dataset2.csv')
df.head(10)

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic,Diastolic
0,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,1,126,83
1,Male,28,Doctor,6.2,6,60,8,Normal Weight,75,10000,1,125,80
2,Male,28,Doctor,6.2,6,60,8,Normal Weight,75,10000,1,125,80
3,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,2,140,90
4,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,2,140,90
5,Male,28,Software Engineer,5.9,4,30,8,Obese,85,3000,0,140,90
6,Male,29,Teacher,6.3,6,40,7,Obese,82,3500,0,140,90
7,Male,29,Doctor,7.8,7,75,6,Normal Weight,70,8000,1,120,80
8,Male,29,Doctor,7.8,7,75,6,Normal Weight,70,8000,1,120,80
9,Male,29,Doctor,7.8,7,75,6,Normal Weight,70,8000,1,120,80


In [3]:
df.shape

(374, 13)

#### Handling Null Values.

In [4]:
df.isna().sum()

Gender                     0
Age                        0
Occupation                 0
Sleep Duration             0
Quality of Sleep           0
Physical Activity Level    0
Stress Level               0
BMI Category               0
Heart Rate                 0
Daily Steps                0
Sleep Disorder             0
Systolic                   0
Diastolic                  0
dtype: int64

* No Null Values

#### Checking Datatypes.

In [5]:
df.dtypes

Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder               int64
Systolic                     int64
Diastolic                    int64
dtype: object

In [6]:
df['Gender'].value_counts()

Gender
Male      189
Female    185
Name: count, dtype: int64

In [7]:
df['Occupation'].value_counts()

Occupation
Nurse                   73
Doctor                  71
Engineer                63
Lawyer                  47
Teacher                 40
Accountant              37
Salesperson             32
Software Engineer        4
Scientist                4
Sales Representative     2
Manager                  1
Name: count, dtype: int64

In [8]:
occupation_counts = df['Occupation'].value_counts()
threshold = 5
df['Occupation'] = df['Occupation'].apply(lambda x: x if occupation_counts[x] >= threshold else 'Others')

In [9]:
df['Occupation'].value_counts()

Occupation
Nurse          73
Doctor         71
Engineer       63
Lawyer         47
Teacher        40
Accountant     37
Salesperson    32
Others         11
Name: count, dtype: int64

In [10]:
df['BMI Category'].value_counts() 

BMI Category
Normal Weight    216
Overweight       148
Obese             10
Name: count, dtype: int64

In [11]:
df['Gender'] = df['Gender'].replace({'Male':1, 'Female':0})
df['Occupation'] = df['Occupation'].replace({'Others':7,'Doctor':1,'Teacher':3,'Nurse':0, 'Engineer':2, 'Accountant':5,'Lawyer':4, 'Salesperson':6})
df['BMI Category'] = df['BMI Category'].replace({'Normal Weight':0, 'Overweight': 1, 'Obese':2})

In [12]:
df.dtypes

Gender                       int64
Age                          int64
Occupation                   int64
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                 int64
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder               int64
Systolic                     int64
Diastolic                    int64
dtype: object

#### Checking target Column.

In [13]:
df['Sleep Disorder'].value_counts()

Sleep Disorder
1    219
2     78
0     77
Name: count, dtype: int64

Thier is imbalance in Our Target Column 'Sleep Disorder'.

#### Statistical Analysis of Data.

In [14]:
df.describe() 

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic,Diastolic
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,0.505348,42.184492,2.564171,7.132086,7.312834,59.171123,5.385027,0.449198,70.165775,6816.844920,1.002674,128.553476,84.649733
std,0.500641,8.673133,2.065911,0.795657,1.196956,20.830804,1.774526,0.549274,4.135676,1617.915679,0.644626,7.748118,6.161611
min,0.000000,27.000000,0.000000,5.800000,4.000000,30.000000,3.000000,0.000000,65.000000,3000.000000,0.000000,115.000000,75.000000
25%,0.000000,35.250000,1.000000,6.400000,6.000000,45.000000,4.000000,0.000000,68.000000,5600.000000,1.000000,125.000000,80.000000
50%,1.000000,43.000000,2.000000,7.200000,7.000000,60.000000,5.000000,0.000000,70.000000,7000.000000,1.000000,130.000000,85.000000
75%,1.000000,50.000000,4.000000,7.800000,8.000000,75.000000,7.000000,1.000000,72.000000,8000.000000,1.000000,135.000000,90.000000
max,1.000000,59.000000,7.000000,8.500000,9.000000,90.000000,8.000000,2.000000,86.000000,10000.000000,2.000000,142.000000,95.000000


#### 6. Checking Highly Correlated Columns.

In [15]:
corr_matrix = df.corr()
threshold = 0.95

high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

high_corr_df = pd.DataFrame(high_corr_pairs, columns=['Feature1', 'Feature2', 'Correlation'])
high_corr_df

,Feature1,Feature2,Correlation
0,Systolic,Diastolic,0.972885


## Splitting the Data into Dependent (y) and Independent (x) Features.

In [16]:
df.shape

(374, 13)

In [17]:
x = df.drop('Sleep Disorder', axis=1)
y = df['Sleep Disorder']

print(x.shape)
print(y.shape)

(374, 12)
(374,)


#### Scaling the Dataset.

In [18]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled_df = pd.DataFrame(x_scaled, columns=x.columns)

class_distribution = y.value_counts(normalize=True)
correlation_with_target = x_scaled_df.corrwith(y)
class_distribution, correlation_with_target.sort_values(ascending=False).head(10)

(Sleep Disorder
 1    0.585561
 2    0.208556
 0    0.205882
 Name: proportion, dtype: float64,
 Physical Activity Level    0.433214
 Daily Steps                0.342098
 Diastolic                  0.310051
 Systolic                   0.241786
 Age                        0.232000
 Heart Rate                 0.205987
 Quality of Sleep           0.179592
 Sleep Duration             0.177552
 BMI Category               0.034457
 Stress Level              -0.036058
 dtype: float64)

In [19]:
x_scaled_df.describe()

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Systolic,Diastolic
count,374.000000,3.740000e+02,3.740000e+02,3.740000e+02,3.740000e+02,3.740000e+02,3.740000e+02,3.740000e+02,3.740000e+02,3.740000e+02,3.740000e+02,3.740000e+02
mean,0.000000,-4.559633e-16,1.709862e-16,1.348891e-15,2.469801e-16,-3.324732e-17,1.994839e-16,7.599388e-17,-1.377389e-15,-9.499234e-17,-9.119265e-16,-8.359326e-16
std,1.001340,1.001340e+00,1.001340e+00,1.001340e+00,1.001340e+00,1.001340e+00,1.001340e+00,1.001340e+00,1.001340e+00,1.001340e+00,1.001340e+00,1.001340e+00
min,-1.010753,-1.753096e+00,-1.242844e+00,-1.676439e+00,-2.771424e+00,-1.402260e+00,-1.345836e+00,-8.188986e-01,-1.250750e+00,-2.362273e+00,-1.751604e+00,-1.568203e+00
25%,-1.010753,-8.006081e-01,-7.581480e-01,-9.213348e-01,-1.098280e+00,-6.812078e-01,-7.815505e-01,-8.188986e-01,-5.243827e-01,-7.531140e-01,-4.592388e-01,-7.556402e-01
50%,0.989361,9.415288e-02,-2.734517e-01,8.547080e-02,-2.617083e-01,3.984423e-02,-2.172650e-01,-8.188986e-01,-4.013793e-02,1.133560e-01,1.869437e-01,5.692287e-02
75%,0.989361,9.023241e-01,6.959410e-01,8.405750e-01,5.748635e-01,7.608963e-01,9.113060e-01,1.004126e+00,4.441068e-01,7.322631e-01,8.331262e-01,8.694859e-01
max,0.989361,1.941401e+00,2.150030e+00,1.721530e+00,1.411435e+00,1.481948e+00,1.475592e+00,2.827150e+00,3.833820e+00,1.970077e+00,1.737782e+00,1.682049e+00


In [20]:
x_scaled_df.head() 

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Systolic,Diastolic
0,0.989361,-1.753096,2.150030,-1.298887,-1.098280,-0.825418,0.347021,1.004126,1.654719,-1.619584,-0.330002,-0.268102
1,0.989361,-1.637643,-0.758148,-1.173036,-1.098280,0.039844,1.475592,-0.818899,1.170474,1.970077,-0.459239,-0.755640
2,0.989361,-1.637643,-0.758148,-1.173036,-1.098280,0.039844,1.475592,-0.818899,1.170474,1.970077,-0.459239,-0.755640
3,0.989361,-1.637643,2.150030,-1.550588,-2.771424,-1.402260,1.475592,2.827150,3.591698,-2.362273,1.479309,0.869486
4,0.989361,-1.637643,2.150030,-1.550588,-2.771424,-1.402260,1.475592,2.827150,3.591698,-2.362273,1.479309,0.869486


In [21]:
y.head()

0    1
1    1
2    1
3    2
4    2
Name: Sleep Disorder, dtype: int64

In [22]:
y.value_counts()

Sleep Disorder
1    219
2     78
0     77
Name: count, dtype: int64

#### Splitting data into Train and Test Data.

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled_df,y, test_size=0.3, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(261, 12)
(113, 12)
(261,)
(113,)


#### Utilizing Random Over Sampler to Achieve Equilibrium in Target Feature Distribution.

In [24]:
ros = RandomOverSampler(random_state=42)
x_train_resampled, y_train_resampled = ros.fit_resample(x_train,y_train)

# Step 3: Check the class distribution after SMOTE
print("Before SMOTE: ", y_train.value_counts())
print("After SMOTE: ", y_train_resampled.value_counts())

Before SMOTE:  Sleep Disorder
1    157
0     53
2     51
Name: count, dtype: int64
After SMOTE:  Sleep Disorder
1    157
2    157
0    157
Name: count, dtype: int64


#### Constructing and Training Diverse Classification Models.

In [25]:
lr = LogisticRegression()
lr.fit(x_train_resampled, y_train_resampled)

LogisticRegression()

In [26]:
dt = DecisionTreeClassifier(max_depth= 20, max_features= 'sqrt', min_samples_leaf= 8, min_samples_split= 10) # max_depth= 20, max_features= 'sqrt', min_samples_leaf= 8, min_samples_split= 10
dt.fit(x_train_resampled, y_train_resampled)

DecisionTreeClassifier(max_depth=20, max_features='sqrt', min_samples_leaf=8,
                       min_samples_split=10)

In [27]:
rf = RandomForestClassifier(n_estimators= 1100, min_samples_split= 20, min_samples_leaf= 25, max_features= 'sqrt', max_depth= None, class_weight= 'balanced', bootstrap= False) # n_estimators= 1100, min_samples_split= 20, min_samples_leaf= 25, max_features= 'sqrt', max_depth= None, class_weight= 'balanced', bootstrap= False
rf.fit(x_train_resampled,y_train_resampled)

RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       min_samples_leaf=25, min_samples_split=20,
                       n_estimators=1100)

In [28]:
gbm = GradientBoostingClassifier(subsample= 0.8, n_estimators= 1000, min_samples_split= 80, min_samples_leaf= 45, max_features= 'sqrt', max_depth= 6, learning_rate= 0.008) # subsample= 0.8, n_estimators= 1000, min_samples_split= 80, min_samples_leaf= 45, max_features= 'sqrt', max_depth= 6, learning_rate= 0.008
gbm.fit(x_train_resampled,y_train_resampled)

GradientBoostingClassifier(learning_rate=0.008, max_depth=6,
                           max_features='sqrt', min_samples_leaf=45,
                           min_samples_split=80, n_estimators=1000,
                           subsample=0.8)

In [29]:
knn = KNeighborsClassifier(metric= 'manhattan', n_neighbors= 6, weights= 'distance')
knn.fit(x_train_resampled,y_train_resampled)

KNeighborsClassifier(metric='manhattan', n_neighbors=6, weights='distance')

#### Evaluation of the Models.

In [30]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1

# Predictions for each model
ypred_rf = rf.predict(x_test)
ypred_dt = dt.predict(x_test)
ypred_gbm = gbm.predict(x_test)
ypred_lr = lr.predict(x_test)
ypred_knn = knn.predict(x_test)

# Evaluate models
results = {
    "Model": ["Decision Tree Classifier", "Random Forest", "Gradient Boosting", "Logistic Regression", "kNN Classifier"],
    "Accuracy": [evaluate_model(y_test, ypred_dt)[0],
                 evaluate_model(y_test, ypred_rf)[0],
                 evaluate_model(y_test, ypred_gbm)[0],
                 evaluate_model(y_test, ypred_lr)[0],
                 evaluate_model(y_test, ypred_knn)[0]],
    "Precision": [evaluate_model(y_test, ypred_dt)[1],
                  evaluate_model(y_test, ypred_rf)[1],
                  evaluate_model(y_test, ypred_gbm)[1],
                  evaluate_model(y_test, ypred_lr)[1],
                  evaluate_model(y_test, ypred_knn)[1]],
    "Recall": [evaluate_model(y_test, ypred_dt)[2],
               evaluate_model(y_test, ypred_rf)[2],
               evaluate_model(y_test, ypred_gbm)[2],
               evaluate_model(y_test, ypred_lr)[2],
               evaluate_model(y_test, ypred_knn)[2]],
    "F1 Score": [evaluate_model(y_test, ypred_dt)[3],
                 evaluate_model(y_test, ypred_rf)[3],
                 evaluate_model(y_test, ypred_gbm)[3],
                 evaluate_model(y_test, ypred_lr)[3],
                 evaluate_model(y_test, ypred_knn)[3]],
    "Training Score": [dt.score(x_train_resampled, y_train_resampled),
                       rf.score(x_train_resampled, y_train_resampled),
                       gbm.score(x_train_resampled, y_train_resampled),
                       lr.score(x_train_resampled, y_train_resampled),
                       knn.score(x_train_resampled, y_train_resampled)],
    "Test Score": [dt.score(x_test, y_test),
                   rf.score(x_test, y_test),
                   gbm.score(x_test, y_test),
                   lr.score(x_test, y_test),
                   knn.score(x_test, y_test)]
}

results_df = pd.DataFrame(results)

In [31]:
results_df

,Model,Accuracy,Precision,Recall,F1 Score,Training Score,Test Score
0,Decision Tree Classifier,0.893805,0.893285,0.893805,0.892840,0.893843,0.893805
1,Random Forest,0.884956,0.888922,0.884956,0.884177,0.902335,0.884956
2,Gradient Boosting,0.902655,0.902980,0.902655,0.901158,0.929936,0.902655
3,Logistic Regression,0.902655,0.905902,0.902655,0.902509,0.902335,0.902655
4,kNN Classifier,0.805310,0.832819,0.805310,0.812208,0.940552,0.805310


In [32]:
from sklearn.metrics import roc_auc_score
import pandas as pd

def compute_roc_auc(model, x_test, y_test):
    # Check if the model has predict_proba
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(x_test)  # Get probability predictions
        # Handle binary and multiclass cases
        if len(y_prob[0]) == 2:  # Binary classification
            auc_score = roc_auc_score(y_test, y_prob[:, 1])
        else:  # Multiclass classification
            auc_score = roc_auc_score(y_test, y_prob, multi_class='ovr')  # Specify multi_class
    else:
        raise ValueError(f"Model {model.__class__.__name__} does not support predict_proba.")
    return auc_score

results = {'Model': [], 'ROC-AUC': []}

models = [dt, rf, gbm, lr, knn]  # Replace with actual trained models (e.g., DecisionTreeClassifier, RandomForestClassifier, etc.)

for model in models:
    try:
        auc_score = compute_roc_auc(model, x_test, y_test)
        results['Model'].append(model.__class__.__name__)
        results['ROC-AUC'].append(auc_score)
    except Exception as e:
        print(f"Error with model {model.__class__.__name__}: {e}")

roc_auc_scores = pd.DataFrame(results)

roc_auc_scores

,Model,ROC-AUC
0,DecisionTreeClassifier,0.912810
1,RandomForestClassifier,0.926296
2,GradientBoostingClassifier,0.917790
3,LogisticRegression,0.925031
4,KNeighborsClassifier,0.885317


In [42]:
! pip install _loss

ERROR: Invalid requirement: '_loss': Expected package name at the start of dependency specifier
    _loss
    ^


In [33]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(x,y)

DecisionTreeClassifier(random_state=42)

In [34]:
rf = RandomForestClassifier(random_state=42)
rf.fit(x,y)

RandomForestClassifier(random_state=42)

In [35]:
gbm = GradientBoostingClassifier(random_state=42)
gbm.fit(x,y)

GradientBoostingClassifier(random_state=42)

In [36]:
lr = LogisticRegression(random_state=42)
lr.fit(x,y)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [37]:
pickle.dump(dt,open("D:\SleepFit Lifestyle and Sleep Analysis\\dt.pkl", "wb"))
pickle.dump(rf,open("D:\SleepFit Lifestyle and Sleep Analysis\\rf.pkl", "wb"))
pickle.dump(gbm,open("D:\SleepFit Lifestyle and Sleep Analysis\\gbm.pkl", "wb"))
pickle.dump(lr,open("D:\SleepFit Lifestyle and Sleep Analysis\\lr.pkl", "wb"))

In [38]:
load_model1 = pickle.load(open("D:\SleepFit Lifestyle and Sleep Analysis\\dt.pkl", "rb"))
load_model2 = pickle.load(open("D:\SleepFit Lifestyle and Sleep Analysis\\rf.pkl", "rb"))
load_model3 = pickle.load(open("D:\SleepFit Lifestyle and Sleep Analysis\\gbm.pkl", "rb"))
load_model4 = pickle.load(open("D:\SleepFit Lifestyle and Sleep Analysis\\lr.pkl", "rb"))

In [39]:
x.head()

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Systolic,Diastolic
0,1,27,7,6.1,6,42,6,1,77,4200,126,83
1,1,28,1,6.2,6,60,8,0,75,10000,125,80
2,1,28,1,6.2,6,60,8,0,75,10000,125,80
3,1,28,7,5.9,4,30,8,2,85,3000,140,90
4,1,28,7,5.9,4,30,8,2,85,3000,140,90


In [40]:
r = y.head(20)
list(r)

[1, 1, 1, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1]

In [41]:
predictions1 = load_model1.predict(x)[:20] 
predictions2 = load_model2.predict(x)[:20] 
predictions3 = load_model3.predict(x)[:20] 
predictions4 = load_model4.predict(x)[:20] 
print(f"Preictions by Decision Tree Classifier :{predictions1},\nPreictions by Random Forest Classifier:{predictions2},\nPreictions by Gradient boost Classifier: {predictions3}, \nPreictions by Logistic Regression: {predictions4}") 

Preictions by Decision Tree Classifier :[1 1 1 2 2 2 0 1 1 1 1 1 1 1 1 1 0 1 0 1],
Preictions by Random Forest Classifier:[1 1 1 2 2 2 0 1 1 1 1 1 1 1 1 1 0 1 0 1],
Preictions by Gradient boost Classifier: [1 1 1 2 2 2 0 1 1 1 1 1 1 1 1 1 0 1 0 1], 
Preictions by Logistic Regression: [1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
